Just a test of published algorithm (#8) https://github.com/kypraios/epiABC

In [4]:
library("rstan")
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)


In [229]:
system('ls',intern=TRUE)

[1] "epiABC"     "main.ipynb" "stan.ipynb"

In [23]:
source('epiABC/R/temporal-functions.R')

In [227]:
priors=c(0.1,0.1)

abcSIR.binned <- function(obs.data.binned, breaks.data, obs.duration, N, epsilon, prior.param, samples) {
   
   # first retrieve the final size of the observed data
   final.size.obs <- length(obs.data.binned)
   
   # matrix to store the posterior samples
   post.samples <- matrix(NA, ncol = 2, nrow=samples)
   
   K = 0
   
   i = 0
   while (i < samples) {  
      # counter
      K = K + 1
      
      # draw from the prior distribution
      beta <- rexp(1, prior.param[1])
      gamma <- rexp(1, prior.param[2])
       
      # simulate data
      sim.data <- simSIR(N, beta, gamma)
      sim.duration <- sim.data$T
      sim.data.binned <- hist(sim.data$removal.times, plot=FALSE, breaks=breaks.data)$counts
      #str(sim.data$removal.times)
      #str(sim.data.binned)
      
      #check if the final size matches the observed data
      d <- sqrt( sum((obs.data.binned - sim.data.binned)^2) + ((obs.duration - sim.duration)/50)^2 )
      if (d < epsilon){
         i <- i + 1
         print(i)
         flush.console() # facilitates print during loop
         post.samples[i,] <- c(beta, gamma)
      }
   }
   print(c(i,K))
   print(post.samples)
}

In [228]:
# vanilla ABC for the Abakaliki Data
abakaliki.removal.times <- cumsum(c(0,13,7,2,3,0,0,1,4,5,3,2,0,2,0,5,3,1,4,0,1,1,1,2,0,1,5,0,5,5))
breaks.abakaliki <- c(0, 13, 26, 39, 52, 65, 78, Inf)
abakaliki.binned.data <- hist(abakaliki.removal.times, breaks=breaks.abakaliki, plot=FALSE)$count

ptm <- proc.time()
out <- abcSIR.binned(obs.data = abakaliki.binned.data,
                     breaks.data = breaks.abakaliki,
                     obs.duration = max(abakaliki.removal.times),
                     N = 120,
                     epsilon = 9.3,
                     prior.param = c(10,10), # if priors are 0.1, fitting takes too long
                     samples = 500)
atm <- proc.time() - ptm
cat("It took", atm[3]/60, "mins")

mean(out[,1])
sd(out[,1])

mean(out[,2])
sd(out[,2])

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

It took 3.57285 mins

[1] 0.09138661

[1] 0.03938797

[1] 0.08414474

[1] 0.03342919

In [226]:
# load stan model into from string instead from file

model_str <- '
data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
model {
  y ~ normal(alpha + beta * x, sigma);
}
'